**Machine Learning from Disaster**

This is my first submission in Kaggle. I have been teaching myself machine learning over the past few months and decided to apply what I have learned into something more practical. I took some bits and pieces of inspiration from other Kernels on the site. 

The model is a simple MLP built with Keras.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#read in database and inspect the headings
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
#we can jump right in to modify categories so they can be understood by the classifier
#we also fill missing values 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train["Sex"] = le.fit_transform(train["Sex"]) 
train["Embarked"] = train["Embarked"].fillna("S") #fill with most common value
train["Embarked"] = le.fit_transform(train["Embarked"])
train["Age"] = train["Age"].fillna(train["Age"].median()) #fill missing values with median age
train.head()

In [ ]:
#select dependent, independent variables for analysis. 
#A priori expectations -> PClass, age and sex likely  significant
X_train = train[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
#X_train = train[["Pclass","Sex","Age"]]
X_train = X_train.values
y_train = train[["Survived"]].values

#standardize data for MLP
#from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
print(X_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
#build, compile and fit the model
#deep networks seem to require a lot of tinkering so we can simplify things with a shallower...
#and wider network. Since database is small we set batch_size = 1 to update weights more frequently
#although there might be a better choice for this
from keras.layers import *
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.callbacks import EarlyStopping
import keras 


model = Sequential()
model.add(Dense(32,
                kernel_initializer = "he_uniform",
                bias_initializer = "zeros",
                #kernel_regularizer=regularizers.l2(0.01),
                activation = "relu",
                input_dim = X_train.shape[1] ))
model.add(Dropout(0.5))

model.add(Dense(32,
                kernel_initializer = "he_uniform",
                bias_initializer = "zeros",
                #kernel_regularizer=regularizers.l2(0.01),
                activation = "relu",
                ))
model.add(Dropout(0.5))

model.add(Dense(32,
                kernel_initializer = "he_uniform",
                bias_initializer = "zeros",
                #kernel_regularizer=regularizers.l2(0.01),
                activation = "relu",
                ))
model.add(Dropout(0.5))

model.add(Dense(1, activation = "sigmoid"))

# opt = optimizers.Adam(lr = 0.0001,decay = 0, beta_1=0.9, beta_2=0.999, 
#                       epsilon = None, amsgrad = True)
lr = 0.01
epochs = 10
decay = lr / epochs
momentum = 0.8
opt = optimizers.SGD(lr=lr, decay=decay, momentum=momentum, nesterov=False)
#opt = optimizers.SGD(lr=0.001)

model.compile(loss = "binary_crossentropy", optimizer = opt, 
              metrics = ["accuracy"])


#track model performance
stop = EarlyStopping(monitor = 'val_loss', patience = epochs, verbose = 1)
history = model.fit(X_train, y_train, epochs = epochs, validation_split = 0.33, shuffle = True, 
        verbose = 1, batch_size = 8, callbacks = [stop])

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#prepare test data for predictions
test = pd.read_csv("../input/test.csv")
test["Sex"] = le.fit_transform(test["Sex"])
test["Embarked"] = test["Embarked"].fillna("S")
test["Embarked"] = le.fit_transform(test["Embarked"])
test["Name"] = le.fit_transform(test["Name"])
#test["Age"] = test["Age"].fillna(test["Age"].median()) 
X_test = test[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].median())
X_test["Fare"] = X_test["Fare"].fillna(X_test["Fare"].mean())
X_test = X_test.values
X_test = preprocessing.scale(X_test)
X_test.shape

In [ ]:
#sigmoid activation gives probabilities of 1 (Surviving). 
#binarizer turns this into appropiate output for competition guidelines
predictions = model.predict(X_test, batch_size=8)
from sklearn.preprocessing import Binarizer
binarizer = Binarizer(threshold = 0.5)
predictions = binarizer.fit_transform(predictions)
predictions = predictions.astype(np.int32)                       

#create submission file
PassengerId = test["PassengerId"]
evaluation = PassengerId.to_frame()
evaluation["Survived"] = predictions
evaluation.to_csv("evaluation_submission.csv", index = False)
"evalutation_submission.csv"